## Load libraries

In [0]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

!pip install -q tf-nightly-2.0-preview
!pip install 'wrapt<1.11.0,>=1.10.0' --force-reinstall

--2019-06-05 13:47:12--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.245.79, 52.73.9.93, 34.199.255.1, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.245.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  38.4MB/s    in 0.4s    

2019-06-05 13:47:13 (38.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
     |████████████████████████████████| 87.1MB 26.0MB/s 
     |████████████████████████████████| 440kB 41.6MB/s 
     |████████████████████████████████| 3.1MB 36.6MB/s 
     |████████████████████████████████| 61kB 21.3MB/s 
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
  Stored in direc

In [0]:
!rm -rf ./logs/

In [0]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp


import matplotlib.pyplot as plt

##Fire-up TensorBoard

[Source 1](https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/)


In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://8966bcc7.ngrok.io


## Load data

In [0]:
dataset = fetch_california_housing()

df = pd.DataFrame(data= np.c_[dataset.data, 100000*dataset.target],
                     columns = dataset.feature_names + ['target'])

X_train, X_test, y_train, y_test = train_test_split(
    df[df['target']<500000].drop(['target'], axis=1).values,
    df[df['target']<500000]['target'],
    test_size=0.2)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

I0605 13:48:09.309810 140647850047360 california_housing.py:114] Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /root/scikit_learn_data


## Train model

[Source 2](https://www.tensorflow.org/tensorboard/r2/hyperparameter_tuning_with_hparams)

In [0]:
HP_NUM_UNITS = hp.HParam('num_units', hp.IntInterval(16, 512))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.3))

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT],
        metrics=[hp.Metric('MeanAbsoluteError', display_name='MAE')],
  )

In [0]:
def train_test_model(run_dir, hparams):
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.keras.activations.relu))
    model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT]))

    model.add(tf.keras.layers.Dense(1))    

    
    model.compile(
        optimizer='Nadam',
        loss=tf.metrics.mean_squared_error,
        metrics=['MeanAbsoluteError', 'RootMeanSquaredError', 'MeanAbsolutePercentageError'],
  )
    
    
    
    tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=run_dir, histogram_freq=1,
                         write_graph=True,
                         write_images=False,
                         profile_batch=0)
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=10,
                   verbose=0,
                   mode='auto',
                   restore_best_weights=True)
    

    model.fit(X_train,
              y_train.values,
              epochs=20000,
              verbose=0,
              batch_size=512,
              validation_split=0.2,
              callbacks=[es, tbCallBack, 
                         hp.KerasCallback('./logs', hparams)]) 
    
    _, mae, rmse, mape = model.evaluate(X_test, y_test.values)
    return rmse


In [0]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(run_dir, hparams)
        tf.summary.scalar('MeanAbsoluteError', accuracy, step=1)

In [0]:
!rm -rf ./logs/

session_num = 0

for num_units in [128, 256, 512]: #HP_NUM_UNITS.domain.values:
    for dropout_rate in [0.1, 0.2, 0.3]:  # (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        
        hparams = {
            HP_NUM_UNITS: num_units,
            HP_DROPOUT: dropout_rate,
        }



        
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)
        session_num += 1


--- Starting trial: run-0
{'num_units': 128, 'dropout': 0.1}
3930/3930 [==============================] - 0s 39us/sample - loss: 3068386664.1588 - MeanAbsoluteError: 39583.3164 - RootMeanSquaredError: 55393.0195 - MeanAbsolutePercentageError: 24.5171
--- Starting trial: run-1
{'num_units': 128, 'dropout': 0.2}
3930/3930 [==============================] - 0s 58us/sample - loss: 3141566985.0545 - MeanAbsoluteError: 40311.0391 - RootMeanSquaredError: 56049.6797 - MeanAbsolutePercentageError: 24.9697
--- Starting trial: run-2
{'num_units': 128, 'dropout': 0.3}
3930/3930 [==============================] - 0s 38us/sample - loss: 3162471577.0137 - MeanAbsoluteError: 40469.2227 - RootMeanSquaredError: 56235.8633 - MeanAbsolutePercentageError: 25.0746
--- Starting trial: run-3
{'num_units': 256, 'dropout': 0.1}
3930/3930 [==============================] - 0s 40us/sample - loss: 3077869147.1959 - MeanAbsoluteError: 39654.2422 - RootMeanSquaredError: 55478.5430 - MeanAbsolutePercentageError: 24.5

# New Section